<header style=" text-align: center; color:blue; font-size:45px">Order Imbalance Based Strategy</header>

<h1 style="color:purple">Author</h1>

  
- Name:Candace Fu Ka Yi                      
- Student ID: 20764468


## Introduction

Many studies have been conducted to describe the relationship between the trade activity(volume) and the price change. Ask (Sell) and Bid (Buy) orders on the border book might signal the direction of market movement. To  model the possible pattern between the order imbalance (between ask and bid sides) and price change, we will build new predictors and check the performance of multiple linear regression in paper trading (backtesting with historical data). 

In this project, you will finish 4 tasks independently. It is not a copy-and-paste project. But if you can complete this project by yourself, it shows that you have passed the entry-level as a data analyst. 

## Data

Totally we have 10 days' data

In [202]:
import pandas as pd
import numpy as np
import os, math, pathlib
from os import listdir
from multiprocessing import Process
import warnings
warnings.filterwarnings('ignore')

In [203]:
stockpath = '/Users/candacefu/Desktop/Assignment/data/'
inputfilename = listdir(stockpath) # get all file names in this folder
inputfilename = sorted(inputfilename)[:] # sort file names 
number = len(inputfilename)
print(number)
print(inputfilename)

10
['20201221_rb2105.csv', '20201222_rb2105.csv', '20201223_rb2105.csv', '20201224_rb2105.csv', '20201225_rb2105.csv', '20210104_rb2105.csv', '20210105_rb2105.csv', '20210106_rb2105.csv', '20210107_rb2105.csv', '20210108_rb2105.csv']


We list file names for these 10 days.  Notice that they are in time order. 

In [204]:
inputfilename[0],inputfilename[-1]

('20201221_rb2105.csv', '20210108_rb2105.csv')

For example, we can read the oldest data as follows.

In [205]:
n=10
data=pd.read_csv(stockpath+inputfilename[-n],index_col=0)
data.head()

,AskPrice,BidPrice,AskVolume,BidVolume
2020-12-21 09:00:00.500,4445.0,4419.0,467.0,87.0
2020-12-21 09:00:01.000,4445.0,4444.0,88.0,67.0
2020-12-21 09:00:01.500,4433.0,4420.0,6.0,462.0
2020-12-21 09:00:02.000,4430.0,4428.0,40.0,9.0
2020-12-21 09:00:02.500,4427.0,4426.0,1.0,81.0


We  read data on Dec 21 , 2020, which has 4 columns. 

## Task 1 - Prepare the Train and Test (10%)

Please use the first 7 days' data to prepare a training data and the remaining (more recent) 3 days' data as the test data. 

In [206]:
data=pd.read_csv(stockpath+inputfilename[0],index_col=0)
train=data

for i in range(6):
    data=pd.read_csv(stockpath+inputfilename[i+1],index_col=0)
    train = pd.concat([train,data]) 
print(train)

                         AskPrice  BidPrice  AskVolume  BidVolume
2020-12-21 09:00:00.500    4445.0    4419.0      467.0       87.0
2020-12-21 09:00:01.000    4445.0    4444.0       88.0       67.0
2020-12-21 09:00:01.500    4433.0    4420.0        6.0      462.0
2020-12-21 09:00:02.000    4430.0    4428.0       40.0        9.0
2020-12-21 09:00:02.500    4427.0    4426.0        1.0       81.0
...                           ...       ...        ...        ...
2021-01-05 14:59:58.000    4398.0    4397.0      120.0      201.0
2021-01-05 14:59:58.500    4398.0    4397.0       92.0      164.0
2021-01-05 14:59:59.000    4398.0    4397.0       84.0       35.0
2021-01-05 14:59:59.500    4398.0    4397.0       47.0       46.0
2021-01-05 15:00:00.000    4398.0    4397.0       84.0       30.0

[236453 rows x 4 columns]


In [207]:
data=pd.read_csv(stockpath+inputfilename[6],index_col=0)
test=data

for i in range(6,9):
    data=pd.read_csv(stockpath+inputfilename[i+1],index_col=0)
    test = pd.concat([test,data])
print(test)

                         AskPrice  BidPrice  AskVolume  BidVolume
2021-01-04 21:00:00.500    4389.0    4388.0        2.0      173.0
2021-01-04 21:00:01.000    4394.0    4393.0       70.0       35.0
2021-01-04 21:00:01.500    4392.0    4391.0       36.0        1.0
2021-01-04 21:00:02.000    4393.0    4392.0       11.0        4.0
2021-01-04 21:00:02.500    4389.0    4388.0       95.0      274.0
...                           ...       ...        ...        ...
2021-01-08 14:59:58.000    4486.0    4485.0       35.0       15.0
2021-01-08 14:59:58.500    4485.0    4484.0        1.0      291.0
2021-01-08 14:59:59.000    4485.0    4484.0        7.0      282.0
2021-01-08 14:59:59.500    4487.0    4486.0      221.0        5.0
2021-01-08 15:00:00.000    4487.0    4486.0      204.0        5.0

[165594 rows x 4 columns]


## Task 2: Build Features (50%)
For both training and test data, build predictors and response

First build mid price
$$Mid_t=\frac{AskPrice_t+BidPrice_t}{2}$$

The frequency of the data is 0.5 second. We want to predict the price change in 10 seconds. Build Y variable 
$$
Y_t=\frac{Mid_{t+1}+Mid_{t+2}+\ldots+Mid_{t+20}}{20}-Mid_t
$$
That is, we calculate the average changes of prices for the next 10 seconds.  

In [208]:
Y = [] 

train['MidPrice'] = (train['AskPrice'] + train['BidPrice'])/2
for i in range(len(train)-20):
    total_20 = 0
    for j in range(1,21):
        total_20 += train.MidPrice[i+j]
    Y_t = (total_20)/20 - train.MidPrice[i]
    Y.append(Y_t)
for i in range(20):
    Y.append(int(0))
train['Y_t'] = Y
print(train)


                         AskPrice  BidPrice  AskVolume  BidVolume  MidPrice  \
2020-12-21 09:00:00.500    4445.0    4419.0      467.0       87.0    4432.0   
2020-12-21 09:00:01.000    4445.0    4444.0       88.0       67.0    4444.5   
2020-12-21 09:00:01.500    4433.0    4420.0        6.0      462.0    4426.5   
2020-12-21 09:00:02.000    4430.0    4428.0       40.0        9.0    4429.0   
2020-12-21 09:00:02.500    4427.0    4426.0        1.0       81.0    4426.5   
...                           ...       ...        ...        ...       ...   
2021-01-05 14:59:58.000    4398.0    4397.0      120.0      201.0    4397.5   
2021-01-05 14:59:58.500    4398.0    4397.0       92.0      164.0    4397.5   
2021-01-05 14:59:59.000    4398.0    4397.0       84.0       35.0    4397.5   
2021-01-05 14:59:59.500    4398.0    4397.0       47.0       46.0    4397.5   
2021-01-05 15:00:00.000    4398.0    4397.0       84.0       30.0    4397.5   

                            Y_t  
2020-12-21 09:00:

Next we build predictors 

#### Volume Order ImBalance

First Build 
$$
OI_t=\delta V_t^B-\delta V_t^A
$$

where

$$
\delta V_t^B=\left\{\begin{array}{cl}0, & \mbox { if } P_t^B<P_{t-1}^B\\V_t^B-V_{t-1}^B, & \mbox { if } P_t^B=P_{t-1}^B\\V_{t}^B, & \mbox { if } P_t^B>P_{t-1}^B\end{array}\right.
$$
and 
$$
\delta V_t^A=\left\{\begin{array}{cl}V_t^A, & \mbox { if } P_t^A<P_{t-1}^A\\V_t^A-V_{t-1}^A, & \mbox { if } P_t^A=P_{t-1}^A\\0, & \mbox { if } P_t^A>P_{t-1}^A\end{array}\right.
$$

where $V_t^A,V_t^B$ are ask and bid volumes, $P_t^A,P_t^B$ are ask and bid prices

Use shift(2),shift(4) ... operators, we  build $OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}$, totally 6 predictors including $OI_t$

In [209]:
OI = [0]
d_V_b = []
d_V_a = []
for i in range(1, len(train)):
    if train.BidPrice[i] < train.BidPrice[i-1]:
        d_V_b.append(0)
    elif train.BidPrice[i] == train.BidPrice[i-1]:
        d_V_b.append(train.BidVolume[i] - train.BidVolume[i-1])
    elif train.BidPrice[i] > train.BidPrice[i-1]:
        d_V_b.append(train.BidVolume[i])
    
    if train.AskPrice[i] < train.AskPrice[i-1]:
        d_V_a.append(train.AskVolume[i])
    elif train.AskPrice[i] == train.AskPrice[i-1]:
        d_V_a.append(train.AskVolume[i] - train.AskVolume[i-1])
    elif train.AskPrice[i] > train.AskPrice[i-1]:
        d_V_a.append(0)
        
for i in range(len(train)-1):
    OI.append(d_V_b[i]-d_V_a[i])

train['OI'] = OI

print(train)

                         AskPrice  BidPrice  AskVolume  BidVolume  MidPrice  \
2020-12-21 09:00:00.500    4445.0    4419.0      467.0       87.0    4432.0   
2020-12-21 09:00:01.000    4445.0    4444.0       88.0       67.0    4444.5   
2020-12-21 09:00:01.500    4433.0    4420.0        6.0      462.0    4426.5   
2020-12-21 09:00:02.000    4430.0    4428.0       40.0        9.0    4429.0   
2020-12-21 09:00:02.500    4427.0    4426.0        1.0       81.0    4426.5   
...                           ...       ...        ...        ...       ...   
2021-01-05 14:59:48.000    4399.0    4398.0      378.0       70.0    4398.5   
2021-01-05 14:59:48.500    4399.0    4398.0      394.0        1.0    4398.5   
2021-01-05 14:59:49.000    4398.0    4397.0       15.0      238.0    4397.5   
2021-01-05 14:59:49.500    4398.0    4397.0       45.0      252.0    4397.5   
2021-01-05 14:59:50.000    4398.0    4397.0        5.0      248.0    4397.5   

                            Y_t     OI  
2020-12-21

In [210]:
for i in range(1, 6):
    train['OI-' + str(2*i) + ''] = train.OI.shift(periods=2*i)
    
train = train.drop(train.index[range(10)])
    
train_OI = train.drop(train.columns[[0,1,2,3,4,5]], axis = 1)
train_Y = train.drop(train.columns[[0,1,2,3,4,6,7,8,9,10,11]], axis = 1)
print(train_OI)
print(train_Y)

                            OI   OI-2   OI-4   OI-6   OI-8  OI-10
2020-12-21 09:00:05.500  187.0  -55.0 -275.0   -1.0   -6.0    0.0
2020-12-21 09:00:06.000 -532.0  -31.0   -9.0  -27.0  -31.0  446.0
2020-12-21 09:00:06.500  321.0  187.0  -55.0 -275.0   -1.0   -6.0
2020-12-21 09:00:07.000 -861.0 -532.0  -31.0   -9.0  -27.0  -31.0
2020-12-21 09:00:07.500    0.0  321.0  187.0  -55.0 -275.0   -1.0
...                        ...    ...    ...    ...    ...    ...
2021-01-05 14:59:48.000  -35.0  147.0  -12.0    2.0  -11.0   -2.0
2021-01-05 14:59:48.500  -85.0  -62.0   10.0    0.0  -20.0  -29.0
2021-01-05 14:59:49.000  -15.0  -35.0  147.0  -12.0    2.0  -11.0
2021-01-05 14:59:49.500  -16.0  -85.0  -62.0   10.0    0.0  -20.0
2021-01-05 14:59:50.000   36.0  -15.0  -35.0  147.0  -12.0    2.0

[236423 rows x 6 columns]
                          Y_t
2020-12-21 09:00:05.500 -5.65
2020-12-21 09:00:06.000 -6.05
2020-12-21 09:00:06.500 -6.45
2020-12-21 09:00:07.000 -5.85
2020-12-21 09:00:07.500 -6.25
.

## Task 3: Make Multiple Linear Regression (20%)

Build multiple linear regression to predict $Y_t$ with $[OI_t, OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}]$ in training data. 

In [211]:
import statsmodels.api as sm
mod = sm.OLS(train_Y, train_OI).fit()
print(mod.summary())

                                 OLS Regression Results                                
Dep. Variable:                    Y_t   R-squared (uncentered):                   0.008
Model:                            OLS   Adj. R-squared (uncentered):              0.008
Method:                 Least Squares   F-statistic:                              326.0
Date:                Tue, 30 Nov 2021   Prob (F-statistic):                        0.00
Time:                        17:57:05   Log-Likelihood:                     -4.0139e+05
No. Observations:              236423   AIC:                                  8.028e+05
Df Residuals:                  236417   BIC:                                  8.029e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Check if there exists multicollinearity and check the residuals to see if the assumptions are satisfied. Discuss your results. 

Using the model you build with training data, get predictions in both training and test data. Comparing Adjusted $R^2$ of the model in both train and test datasets, how to evaluate your model? Overfitting? Good Signal?

In [212]:
Y = []


test['MidPrice'] = (test['AskPrice'] + test['BidPrice'])/2
for i in range(len(test)-20):
    total_20 = 0
    for j in range(1,21):
        total_20 += test.MidPrice[i+j]
    Y_t = (total_20)/20 - test.MidPrice[i]
    Y.append(Y_t)
for i in range(20):
    Y.append(int(0))
test['Y_t'] = Y
OI = [0]
d_V_b = []
d_V_a = []
for i in range(1, len(test)):
    if test.BidPrice[i] < test.BidPrice[i-1]:
        d_V_b.append(0)
    elif test.BidPrice[i] == test.BidPrice[i-1]:
        d_V_b.append(test.BidVolume[i] - test.BidVolume[i-1])
    elif test.BidPrice[i] > test.BidPrice[i-1]:
        d_V_b.append(test.BidVolume[i])
    
    if test.AskPrice[i] < test.AskPrice[i-1]:
        d_V_a.append(test.AskVolume[i])
    elif test.AskPrice[i] == test.AskPrice[i-1]:
        d_V_a.append(test.AskVolume[i] - test.AskVolume[i-1])
    elif test.AskPrice[i] > test.AskPrice[i-1]:
        d_V_a.append(0)
        
for i in range(len(test)-1):
    OI.append(d_V_b[i]-d_V_a[i])

test['OI'] = OI


for i in range(1, 6):
    test['OI-' + str(2*i) + ''] = test.OI.shift(periods=2*i)
test = test.drop(test.index[range(10)])
test_OI = test.drop(test.columns[[0,1,2,3,4,5]], axis = 1)
test_Y = test.drop(test.columns[[0,1,2,3,4,6,7,8,9,10,11]], axis = 1)
regr.fit(test_OI, test_Y)

mod = sm.OLS(test_Y, test_OI).fit()
print(mod.summary())

                                 OLS Regression Results                                
Dep. Variable:                    Y_t   R-squared (uncentered):                   0.019
Model:                            OLS   Adj. R-squared (uncentered):              0.019
Method:                 Least Squares   F-statistic:                              542.9
Date:                Tue, 30 Nov 2021   Prob (F-statistic):                        0.00
Time:                        17:57:55   Log-Likelihood:                     -2.0738e+05
No. Observations:              165584   AIC:                                  4.148e+05
Df Residuals:                  165578   BIC:                                  4.148e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## Task 4: Nonlinear regression  and Subset Slection (20%) 

Build interaction terms and square terms  out of $[OI_t, OI_{t-2},OI_{t-4},OI_{t-6},OI_{t-8}, OI_{t-10}]$.   Apply best subset selection with indirected criterion (Adjusted Rsquare).  Evaluate your model in Test dataset. 